In [1]:
import pandas as pd
import calendar


In [44]:
def Get_days_from_month(date):
    year = date.year
    month = date.month
    # year, month, day = map(int, date.split('-'))

    # 获取该月的天数
    days_in_month = calendar.monthrange(year, month)[1]

    return days_in_month

In [45]:
guide_monthly_flow = pd.read_excel('H:/DataSpace/guide_monthflow1950-2018.xls')
# remove 站名 column
guide_monthly_flow = guide_monthly_flow.drop('站名', axis=1)
# rename the dates column name to DATE
guide_monthly_flow.columns = ['date', 'flow(m^3/s)']
guide_monthly_flow.to_csv('../data/guide_monthly_flow.csv', index=False)
guide_monthly_flow['days'] = guide_monthly_flow['date'].apply(Get_days_from_month)
guide_monthly_flow['runoff(10^8m^3)'] = guide_monthly_flow['flow(m^3/s)']*guide_monthly_flow['days']*24*3600/100000000
guide_annual_runoff = guide_monthly_flow.loc[:,['date','runoff(10^8m^3)']]
guide_annual_runoff['date']=pd.to_datetime(guide_annual_runoff['date'])
guide_annual_runoff.set_index('date',inplace=True)
guide_annual_runoff = guide_annual_runoff.resample('YE').sum()
guide_annual_runoff.to_csv('../data/guide_annual_runoff.csv')

In [47]:
tangnaihai_monthly_flow = pd.read_excel('H:/DataSpace/唐乃亥月径流1950-2018.xls')
tangnaihai_monthly_flow = tangnaihai_monthly_flow.iloc[:,1:3]
tangnaihai_monthly_flow.columns = ['date','flow(m^3/s)']
tangnaihai_monthly_flow.to_csv('../data/tangnaihai_monthly_flow.csv',index=False)
tangnaihai_monthly_flow['days'] = tangnaihai_monthly_flow['date'].apply(Get_days_from_month)
tangnaihai_monthly_flow['runoff(10^8m^3)'] = tangnaihai_monthly_flow['flow(m^3/s)']*tangnaihai_monthly_flow['days']*24*3600/100000000
tangnaihai_annual_runoff = tangnaihai_monthly_flow.loc[:,['date','runoff(10^8m^3)']]
tangnaihai_annual_runoff['date']=pd.to_datetime(tangnaihai_annual_runoff['date'])
tangnaihai_annual_runoff.set_index('date',inplace=True)
tangnaihai_annual_runoff = tangnaihai_annual_runoff.resample('YE').sum()
tangnaihai_annual_runoff.to_csv('../data/tangnaihai_annual_runoff.csv')


In [48]:
xunhua_monthly_flow = pd.read_excel('H:/DataSpace/循化站月径流1957-2018.xls')
xunhua_monthly_flow = xunhua_monthly_flow.iloc[:,1:3]
xunhua_monthly_flow.columns = ['date','flow(m^3/s)']
xunhua_monthly_flow.to_csv('../data/xunhua_monthly_flow.csv',index=False)
xunhua_monthly_flow['days'] = xunhua_monthly_flow['date'].apply(Get_days_from_month)
xunhua_monthly_flow['runoff(10^8m^3)'] = xunhua_monthly_flow['flow(m^3/s)']*xunhua_monthly_flow['days']*24*3600/100000000
xunhua_annual_runoff = xunhua_monthly_flow.loc[:,['date','runoff(10^8m^3)']]
xunhua_annual_runoff['date']=pd.to_datetime(xunhua_annual_runoff['date'])
xunhua_annual_runoff.set_index('date',inplace=True)
xunhua_annual_runoff = xunhua_annual_runoff.resample('YE').sum()
xunhua_annual_runoff.to_csv('../data/xunhua_annual_runoff.csv')

In [7]:
guide_natural = pd.read_excel('H:/连亚妮论文数据/第三章/贵德站多种方法还原结果对比.xls',sheet_name='月径流还原结果',parse_dates=['dates'],index_col='dates')
xunhua_natural = pd.read_excel('H:/连亚妮论文数据/第三章/循化站多种方法还原结果对比.xls',sheet_name='月径流还原结果',parse_dates=['dates'],index_col='dates')

guide_natural.columns = ['flow','VIF-MLR','VIF-LSTM']                      
xunhua_natural.columns = ['flow','VIF-MLR','VIF-LSTM']

guide_natural.index.name = 'date'

xunhua_natural.index.name = 'date'

guide_natural.to_csv('../data/guide_naturalized_monthly_flow.csv')
xunhua_natural.to_csv('../data/xunhua_naturalized_monthly_flow.csv')


In [1]:
import pandas as pd
start = '1957-01-01'
end = '2018-12-31'

date_index = pd.date_range(start, end, freq='ME')

era5l_guide = pd.read_csv('../data/guide_era5l_month1957-2019.csv',parse_dates=['date'],index_col='date')
era5l_guide = era5l_guide.loc[start:end,['v10','snowc','sd','ssro']]
era5l_guide.set_index(date_index,inplace=True)

mete_guide_guide = pd.read_csv('../data/guide_monthly_mete.csv',parse_dates=['DATE'],index_col='DATE')
mete_guide_guide = mete_guide_guide.loc[start:end,['P208(mm)','P820(mm)']]
mete_guide_guide.columns=['P208_gd','P820_gd']
mete_guide_guide.set_index(date_index,inplace=True) 
mete_guide_guide.index.name='date'

mete_guide_gonghe = pd.read_csv('../data/gonghe_monthly_mete.csv',parse_dates=['DATE'],index_col='DATE')
mete_guide_gonghe = mete_guide_gonghe.loc[start:end,['P820(mm)']]
mete_guide_gonghe.columns=['P820_gh']
mete_guide_gonghe.set_index(date_index,inplace=True) 
mete_guide_gonghe.index.name='date'

tnhIN_guide = pd.read_csv('../data/tangnaihai_monthly_flow.csv',parse_dates=['date'],index_col='date')
tnhIN_guide = tnhIN_guide.loc[start:end]
tnhIN_guide.set_index(date_index,inplace=True) 
tnhIN_guide.columns=['tnh_flow']

guide = pd.read_csv('../data/guide_monthly_flow.csv',parse_dates=['date'],index_col='date')
guide = guide.loc[start:end]
guide.set_index(date_index,inplace=True) 
guide.columns=['flow']

full_data_guide = pd.concat([era5l_guide,mete_guide_guide,mete_guide_gonghe,tnhIN_guide,guide],axis=1)
full_data_guide.index.name='date'
full_data_guide.to_csv('../data/full_modeling_data_guide.csv')

In [12]:
import pandas as pd
start = '1957-01-01'
end = '2018-12-31'

date_index = pd.date_range(start, end, freq='ME')

era5l_xunhua = pd.read_csv('../data/xunhua_era5l_month1957-2019.csv')
era5l_xunhua.index = pd.date_range('1957-01-01','2019-12-31',freq='ME')
era5l_xunhua = era5l_xunhua.loc[start:end,['v10','sd','ssro','sro','smlt']]
era5l_xunhua.set_index(date_index,inplace=True)

mete_xunhua_guide = pd.read_csv('../data/guide_monthly_mete.csv',parse_dates=['DATE'],index_col='DATE')
mete_xunhua_guide = mete_xunhua_guide.loc[start:end,['P208(mm)','P820(mm)']]
mete_xunhua_guide.columns=['P208_gd','P820_gd']
mete_xunhua_guide.set_index(date_index,inplace=True) 
mete_xunhua_guide.index.name='date'

mete_xunhua_gonghe = pd.read_csv('../data/gonghe_monthly_mete.csv',parse_dates=['DATE'],index_col='DATE')
mete_xunhua_gonghe = mete_xunhua_gonghe.loc[start:end,['P820(mm)']]
mete_xunhua_gonghe.columns=['P820_gh']
mete_xunhua_gonghe.set_index(date_index,inplace=True) 
mete_xunhua_gonghe.index.name='date'

gdIN_xunhua_pre = pd.read_csv('../data/guide_monthly_flow.csv',parse_dates=['date'],index_col='date')
gdIN_xunhua_pre = gdIN_xunhua_pre.loc['1957-01-01':'1985-12-31']
gdIN_xunhua_pre.set_index(pd.date_range('1957-01-01','1985-12-31',freq='ME'),inplace=True) 
gdIN_xunhua_pre.columns=['gd_flow']
gdIN_xunhua_infl = pd.read_csv('../data/guide_monthly_naturalized_flow_viflstm.csv',parse_dates=['date'],index_col='date')
gdIN_xunhua_infl = gdIN_xunhua_infl.loc['1986-01-01':'2018-12-31']
gdIN_xunhua_infl = gdIN_xunhua_infl['VIF_LSTM']
gdIN_xunhua_infl.name= 'gd_flow'
gdIn_xunhua = pd.concat([gdIN_xunhua_pre,gdIN_xunhua_infl],axis=0) 
gdIn_xunhua


tnhIN_xunhua = pd.read_csv('../data/tangnaihai_monthly_flow.csv',parse_dates=['date'],index_col='date')
tnhIN_xunhua = tnhIN_xunhua.loc[start:end]
tnhIN_xunhua.set_index(date_index,inplace=True) 
tnhIN_xunhua.columns=['tnh_flow']

xunhua = pd.read_csv('../data/xunhua_monthly_flow.csv',parse_dates=['date'],index_col='date')
xunhua = xunhua.loc[start:end]
xunhua.set_index(date_index,inplace=True) 
xunhua.columns=['flow']

full_data_xunhua = pd.concat([era5l_xunhua,mete_xunhua_guide,mete_xunhua_gonghe,tnhIN_xunhua,xunhua],axis=1)
full_data_xunhua.index.name='date'
full_data_xunhua
full_data_xunhua.to_csv('../data/full_modeling_data_xunhua.csv')